Execute everything with Ctrl + F9

# Read in Excel Table

In [ ]:
#Packages
import pandas as pd
import glob
from datetime import timedelta
import calendar

#Access to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Define the month

In [ ]:
this_year = input("For which year do you want to run the analysis? ")
this_month = input("And what is the month? ")
time_frame = this_year + "/" + this_month

Read in names of all tables

In [ ]:
filenames = glob.glob("/content/drive/MyDrive/Performance metrics/"+time_frame+"/*")
filenames

['/content/drive/MyDrive/Performance metrics/2022/January/detailed_report_20220102.csv',
 '/content/drive/MyDrive/Performance metrics/2022/January/detailed_report_20220103.csv',
 '/content/drive/MyDrive/Performance metrics/2022/January/detailed_report_20220104.csv',
 '/content/drive/MyDrive/Performance metrics/2022/January/detailed_report_20220105.csv',
 '/content/drive/MyDrive/Performance metrics/2022/January/detailed_report_20220106.csv',
 '/content/drive/MyDrive/Performance metrics/2022/January/detailed_report_20220107.csv',
 '/content/drive/MyDrive/Performance metrics/2022/January/detailed_report_20220108.csv',
 '/content/drive/MyDrive/Performance metrics/2022/January/detailed_report_20220109.csv',
 '/content/drive/MyDrive/Performance metrics/2022/January/detailed_report_20220101.csv',
 '/content/drive/MyDrive/Performance metrics/2022/January/detailed_report_20220110.csv',
 '/content/drive/MyDrive/Performance metrics/2022/January/detailed_report_20220111.csv',
 '/content/drive/MyDr

Read in all tables by their filenames and merge them together to one big table

In [ ]:
tables = [pd.read_csv(f, sep=",") for f in filenames]
df = pd.concat(tables, ignore_index=True)

In [ ]:
df

,Day,User,Time Entry,Project,Project Code,Client,Tags,Work Type,Start Time,End Time,Duration,Issue Id,Link
0,2022-01-02,Santiago Salazar,Controller Offline,NaN,NaN,NaN,NaN,NaN,06:13,06:31,0:18:00,#261285,"=HYPERLINK(""https://ezlosupport.zendesk.com/ag..."
1,2022-01-02,Santiago Salazar,RMA VERA - December 2021,NaN,NaN,NaN,NaN,NaN,06:31,09:11,2:40:00,#261929,"=HYPERLINK(""https://ezlosupport.zendesk.com/ag..."
2,2022-01-02,Santiago Salazar,Voicemail from: Rick Madden,NaN,NaN,NaN,NaN,NaN,09:11,09:24,0:13:00,#263346,"=HYPERLINK(""https://ezlosupport.zendesk.com/ag..."
3,2022-01-02,Santiago Salazar,Voicemail from: Caller +1 (469) 235-5442,NaN,NaN,NaN,NaN,NaN,09:24,09:35,0:11:00,#263351,"=HYPERLINK(""https://ezlosupport.zendesk.com/ag..."
4,2022-01-02,Santiago Salazar,Voicemail from: Caller +1 (509) 995-3432,NaN,NaN,NaN,NaN,NaN,09:35,09:40,0:05:00,#263352,"=HYPERLINK(""https://ezlosupport.zendesk.com/ag..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3561,2022-01-31,Juan Pelaez,#265360 Voicemail from: Caller +1 (917) 378-9613,Ticketing metrics T1,NaN,NaN,#265360 Voicemail from: Caller +1 (917) 378-9613,NaN,20:50,21:18,0:28:00,#265360,"=HYPERLINK(""https://ezlosupport.zendesk.com/ag..."
3562,2022-01-31,Stefany Cantero,Vera Plus has lost connection to ALL Zigbee de...,Ticketing metrics T1,NaN,NaN,NaN,NaN,21:14,21:30,0:16:00,#264594,"=HYPERLINK(""https://ezlosupport.zendesk.com/ag..."
3563,2022-01-31,Juan Pelaez,looses connection,Ticketing metrics T1,NaN,NaN,looses connection,NaN,21:18,21:29,0:11:00,#265124,"=HYPERLINK(""https://ezlosupport.zendesk.com/ag..."
3564,2022-01-31,Juan Pelaez,#265396 Voicemail from: Caller +1 (862) 703-8518,Ticketing metrics T1,NaN,NaN,#265396 Voicemail from: Caller +1 (862) 703-8518,NaN,21:29,21:45,0:16:00,#265396,"=HYPERLINK(""https://ezlosupport.zendesk.com/ag..."


# Transform Table

Delete useless columns

In [ ]:
df = df.drop(['Project Code', 'Client', 'Tags', 'Work Type'], axis=1)
df.head()

,Day,User,Time Entry,Project,Start Time,End Time,Duration,Issue Id,Link
0,2022-01-02,Santiago Salazar,Controller Offline,NaN,06:13,06:31,0:18:00,#261285,"=HYPERLINK(""https://ezlosupport.zendesk.com/ag..."
1,2022-01-02,Santiago Salazar,RMA VERA - December 2021,NaN,06:31,09:11,2:40:00,#261929,"=HYPERLINK(""https://ezlosupport.zendesk.com/ag..."
2,2022-01-02,Santiago Salazar,Voicemail from: Rick Madden,NaN,09:11,09:24,0:13:00,#263346,"=HYPERLINK(""https://ezlosupport.zendesk.com/ag..."
3,2022-01-02,Santiago Salazar,Voicemail from: Caller +1 (469) 235-5442,NaN,09:24,09:35,0:11:00,#263351,"=HYPERLINK(""https://ezlosupport.zendesk.com/ag..."
4,2022-01-02,Santiago Salazar,Voicemail from: Caller +1 (509) 995-3432,NaN,09:35,09:40,0:05:00,#263352,"=HYPERLINK(""https://ezlosupport.zendesk.com/ag..."


Delete not working related entries

In [ ]:
#Drop Rows with Empty Entries
df.dropna(subset = ['Time Entry'], inplace=True)

#Drop Rows containing one of the following words
filters = ["unch", "reakfast", "reak", "athroom", "inner", "lean"]
df = df[df['Time Entry'].apply(lambda time_entry: all(filter not in time_entry for filter in filters))]
df.head()

,Day,User,Time Entry,Project,Start Time,End Time,Duration,Issue Id,Link
0,2022-01-02,Santiago Salazar,Controller Offline,NaN,06:13,06:31,0:18:00,#261285,"=HYPERLINK(""https://ezlosupport.zendesk.com/ag..."
1,2022-01-02,Santiago Salazar,RMA VERA - December 2021,NaN,06:31,09:11,2:40:00,#261929,"=HYPERLINK(""https://ezlosupport.zendesk.com/ag..."
2,2022-01-02,Santiago Salazar,Voicemail from: Rick Madden,NaN,09:11,09:24,0:13:00,#263346,"=HYPERLINK(""https://ezlosupport.zendesk.com/ag..."
3,2022-01-02,Santiago Salazar,Voicemail from: Caller +1 (469) 235-5442,NaN,09:24,09:35,0:11:00,#263351,"=HYPERLINK(""https://ezlosupport.zendesk.com/ag..."
4,2022-01-02,Santiago Salazar,Voicemail from: Caller +1 (509) 995-3432,NaN,09:35,09:40,0:05:00,#263352,"=HYPERLINK(""https://ezlosupport.zendesk.com/ag..."


Count amount of Phone Calls

In [ ]:
phonecalls = []
for entry in df["Time Entry"]:
  if "hone" in entry or "oice" in entry:
    phonecalls.append(1)
  else:
    phonecalls.append(0)    

df["Phonecalls"] = phonecalls
df.head()

,Day,User,Time Entry,Project,Start Time,End Time,Duration,Issue Id,Link,Phonecalls
0,2022-01-02,Santiago Salazar,Controller Offline,NaN,06:13,06:31,0:18:00,#261285,"=HYPERLINK(""https://ezlosupport.zendesk.com/ag...",0
1,2022-01-02,Santiago Salazar,RMA VERA - December 2021,NaN,06:31,09:11,2:40:00,#261929,"=HYPERLINK(""https://ezlosupport.zendesk.com/ag...",0
2,2022-01-02,Santiago Salazar,Voicemail from: Rick Madden,NaN,09:11,09:24,0:13:00,#263346,"=HYPERLINK(""https://ezlosupport.zendesk.com/ag...",1
3,2022-01-02,Santiago Salazar,Voicemail from: Caller +1 (469) 235-5442,NaN,09:24,09:35,0:11:00,#263351,"=HYPERLINK(""https://ezlosupport.zendesk.com/ag...",1
4,2022-01-02,Santiago Salazar,Voicemail from: Caller +1 (509) 995-3432,NaN,09:35,09:40,0:05:00,#263352,"=HYPERLINK(""https://ezlosupport.zendesk.com/ag...",1


Transform formation of Times

In [ ]:
df["Day"] = pd.to_datetime(df["Day"])
df['Duration'] = pd.to_timedelta(df.Duration)
df.head()

,Day,User,Time Entry,Project,Start Time,End Time,Duration,Issue Id,Link,Phonecalls
0,2022-01-02,Santiago Salazar,Controller Offline,NaN,06:13,06:31,0 days 00:18:00,#261285,"=HYPERLINK(""https://ezlosupport.zendesk.com/ag...",0
1,2022-01-02,Santiago Salazar,RMA VERA - December 2021,NaN,06:31,09:11,0 days 02:40:00,#261929,"=HYPERLINK(""https://ezlosupport.zendesk.com/ag...",0
2,2022-01-02,Santiago Salazar,Voicemail from: Rick Madden,NaN,09:11,09:24,0 days 00:13:00,#263346,"=HYPERLINK(""https://ezlosupport.zendesk.com/ag...",1
3,2022-01-02,Santiago Salazar,Voicemail from: Caller +1 (469) 235-5442,NaN,09:24,09:35,0 days 00:11:00,#263351,"=HYPERLINK(""https://ezlosupport.zendesk.com/ag...",1
4,2022-01-02,Santiago Salazar,Voicemail from: Caller +1 (509) 995-3432,NaN,09:35,09:40,0 days 00:05:00,#263352,"=HYPERLINK(""https://ezlosupport.zendesk.com/ag...",1


# Create New Dataframe

Add Phonecalls

In [ ]:
new_df = df.groupby('User')[['Phonecalls']].sum()
new_df

,Phonecalls
User,
Alvaro Ochoa,263
Brayan Arbelaez,24
Catalina Granados,10
Daniel Garcia,103
Daniela Puerta Aguirre,0
Juan Pelaez,89
Leandro Arango Usuga,120
Leonardo Soto,0
Oscar Morales,92


Add Mails

In [ ]:
#mails are difference between number of all time entries and number of phone calls
temp_df = df.groupby('User')[['Time Entry']].count()
new_df["Mails"] = temp_df["Time Entry"] - new_df["Phonecalls"]
new_df

,Phonecalls,Mails
User,,
Alvaro Ochoa,263,318
Brayan Arbelaez,24,91
Catalina Granados,10,2
Daniel Garcia,103,164
Daniela Puerta Aguirre,0,4
Juan Pelaez,89,441
Leandro Arango Usuga,120,636
Leonardo Soto,0,6
Oscar Morales,92,521


Add Duration

In [ ]:
temp_df = df.groupby(['User'])[['Duration']].sum()
#check that times dont exceed maximum alowed working time
new_df['Duration'] = temp_df['Duration']
new_df

,Phonecalls,Mails,Duration
User,,,
Alvaro Ochoa,263,318,6 days 15:44:00
Brayan Arbelaez,24,91,5 days 14:01:00
Catalina Granados,10,2,0 days 04:20:00
Daniel Garcia,103,164,6 days 10:16:00
Daniela Puerta Aguirre,0,4,0 days 04:48:00
Juan Pelaez,89,441,6 days 08:00:00
Leandro Arango Usuga,120,636,6 days 13:52:00
Leonardo Soto,0,6,0 days 03:35:00
Oscar Morales,92,521,6 days 15:01:00


Add Working Days

In [ ]:
#df.User.unique()
temp_df = df.groupby(['User'])[['Day']].nunique()
new_df['Working Days'] = temp_df['Day']
new_df

,Phonecalls,Mails,Duration,Working Days
User,,,,
Alvaro Ochoa,263,318,6 days 15:44:00,20
Brayan Arbelaez,24,91,5 days 14:01:00,18
Catalina Granados,10,2,0 days 04:20:00,1
Daniel Garcia,103,164,6 days 10:16:00,19
Daniela Puerta Aguirre,0,4,0 days 04:48:00,2
Juan Pelaez,89,441,6 days 08:00:00,19
Leandro Arango Usuga,120,636,6 days 13:52:00,21
Leonardo Soto,0,6,0 days 03:35:00,2
Oscar Morales,92,521,6 days 15:01:00,20


Add Days Off

In [ ]:
tot_days = df['Day'].nunique()
new_df['Days Off'] = tot_days - new_df['Working Days']
new_df

,Phonecalls,Mails,Duration,Working Days,Days Off
User,,,,,
Alvaro Ochoa,263,318,6 days 15:44:00,20,10
Brayan Arbelaez,24,91,5 days 14:01:00,18,12
Catalina Granados,10,2,0 days 04:20:00,1,29
Daniel Garcia,103,164,6 days 10:16:00,19,11
Daniela Puerta Aguirre,0,4,0 days 04:48:00,2,28
Juan Pelaez,89,441,6 days 08:00:00,19,11
Leandro Arango Usuga,120,636,6 days 13:52:00,21,9
Leonardo Soto,0,6,0 days 03:35:00,2,28
Oscar Morales,92,521,6 days 15:01:00,20,10


Add Total Working Time

In [ ]:
#total hours is number of working days * 8 hours
new_df ['Total Time'] = (new_df['Working Days'] * 8).astype(str) + 'h'
new_df ['Total Time'] = pd.to_timedelta(new_df ['Total Time'])
new_df

,Phonecalls,Mails,Duration,Working Days,Days Off,Total Time
User,,,,,,
Alvaro Ochoa,263,318,6 days 15:44:00,20,10,6 days 16:00:00
Brayan Arbelaez,24,91,5 days 14:01:00,18,12,6 days 00:00:00
Catalina Granados,10,2,0 days 04:20:00,1,29,0 days 08:00:00
Daniel Garcia,103,164,6 days 10:16:00,19,11,6 days 08:00:00
Daniela Puerta Aguirre,0,4,0 days 04:48:00,2,28,0 days 16:00:00
Juan Pelaez,89,441,6 days 08:00:00,19,11,6 days 08:00:00
Leandro Arango Usuga,120,636,6 days 13:52:00,21,9,7 days 00:00:00
Leonardo Soto,0,6,0 days 03:35:00,2,28,0 days 16:00:00
Oscar Morales,92,521,6 days 15:01:00,20,10,6 days 16:00:00


Check that Working Time doesnt exceed the total time

In [ ]:
new_df['Duration'] = new_df[['Duration','Total Time']].min(axis=1)
new_df

,Phonecalls,Mails,Duration,Working Days,Days Off,Total Time
User,,,,,,
Alvaro Ochoa,263,318,6 days 15:44:00,20,10,6 days 16:00:00
Brayan Arbelaez,24,91,5 days 14:01:00,18,12,6 days 00:00:00
Catalina Granados,10,2,0 days 04:20:00,1,29,0 days 08:00:00
Daniel Garcia,103,164,6 days 08:00:00,19,11,6 days 08:00:00
Daniela Puerta Aguirre,0,4,0 days 04:48:00,2,28,0 days 16:00:00
Juan Pelaez,89,441,6 days 08:00:00,19,11,6 days 08:00:00
Leandro Arango Usuga,120,636,6 days 13:52:00,21,9,7 days 00:00:00
Leonardo Soto,0,6,0 days 03:35:00,2,28,0 days 16:00:00
Oscar Morales,92,521,6 days 15:01:00,20,10,6 days 16:00:00


Calculate Efficiency

In [ ]:
new_df['Efficiency'] = (new_df['Duration'] / new_df['Total Time'])
new_df['Efficiency'] = round(new_df['Efficiency']*100,2)
new_df['Efficiency'] = new_df['Efficiency'].apply(lambda x: str(x)  + '%')
new_df

,Phonecalls,Mails,Duration,Working Days,Days Off,Total Time,Efficiency
User,,,,,,,
Alvaro Ochoa,263,318,6 days 15:44:00,20,10,6 days 16:00:00,99.83%
Brayan Arbelaez,24,91,5 days 14:01:00,18,12,6 days 00:00:00,93.07%
Catalina Granados,10,2,0 days 04:20:00,1,29,0 days 08:00:00,54.17%
Daniel Garcia,103,164,6 days 08:00:00,19,11,6 days 08:00:00,100.0%
Daniela Puerta Aguirre,0,4,0 days 04:48:00,2,28,0 days 16:00:00,30.0%
Juan Pelaez,89,441,6 days 08:00:00,19,11,6 days 08:00:00,100.0%
Leandro Arango Usuga,120,636,6 days 13:52:00,21,9,7 days 00:00:00,93.97%
Leonardo Soto,0,6,0 days 03:35:00,2,28,0 days 16:00:00,22.4%
Oscar Morales,92,521,6 days 15:01:00,20,10,6 days 16:00:00,99.39%


Get Date as String to use it as new filename

In [ ]:
month = calendar.month_name[df['Day'][0].month]
year = df['Day'][0].year
new_filename = 'resumed_report_' + str(year) + '_' + month + '.csv'
new_filename

'resumed_report_2022_January.csv'

Save it as an csv

In [ ]:
new_df.to_csv(new_filename, sep=",")

# Create Group Table

In [ ]:
#entries are sums of new_df
group_df = pd.DataFrame(new_df[['Duration','Total Time','Working Days', 'Days Off']].sum())
#transpose it
group_df = group_df.T
#calculate efficiency
group_df['Efficiency'] = group_df['Duration'] / group_df['Total Time']
group_df['Efficiency'] = round(group_df['Efficiency']*100,2)
group_df['Efficiency'] = group_df['Efficiency'].apply(lambda x: str(x)  + '%')
#set Month as index-column
group_df ['Month'] = 'September'
group_df = group_df.groupby(['Month'])[['Duration','Total Time','Working Days', 'Days Off','Efficiency']].sum()

group_df

,Duration,Total Time,Working Days,Days Off,Efficiency
Month,,,,,
September,50 days 00:58:00,52 days 08:00:00,157,173,95.62%


Create Filename

In [ ]:
group_filename = new_filename.replace("resumed","group")
group_filename

'group_report_2022_January.csv'

Save it as an csv

In [ ]:
group_df.to_csv(group_filename, sep=",")